![Slide One](https://d3ddy8balm3goa.cloudfront.net/rbc-llm-workshop/1.svg)

![Divider Image](https://d3ddy8balm3goa.cloudfront.net/mlops-rag-workshop/divider-2.excalidraw.svg)
![Slide Two](https://d3ddy8balm3goa.cloudfront.net/rbc-llm-workshop/2.svg)

![Divider Image](https://d3ddy8balm3goa.cloudfront.net/mlops-rag-workshop/divider-2.excalidraw.svg)
![Slide Three](https://d3ddy8balm3goa.cloudfront.net/rbc-llm-workshop/3.svg)

![Divider Image](https://d3ddy8balm3goa.cloudfront.net/mlops-rag-workshop/divider-2.excalidraw.svg)
![Slide Four](https://d3ddy8balm3goa.cloudfront.net/rbc-llm-workshop/4-updated.svg)

## Example: A Gang of LLMs Tell A Story

In [ ]:
# INSTALL LLM INTEGRATION PACKAGES
# %pip install llama-index-llms-openai -q
# %pip install llama-index-llms-cohere -q
# %pip install llama-index-llms-anthropic -q
# %pip install llama-index-llms-mistralai -q
# %pip install llama-index-vector-stores-qdrant -q
# %pip install llama-index-agent-openai -q
# %pip install llama-index-agent-introspective -q
# %pip install google-api-python-client -q
# %pip install llama-index-program-openai -q
# %pip install llama-index-readers-file -q

In [ ]:
from llama_index.llms.anthropic import Anthropic
from llama_index.llms.cohere import Cohere
from llama_index.llms.mistralai import MistralAI
from llama_index.llms.openai import OpenAI

anthropic_llm = Anthropic(model="claude-3-opus-20240229")
cohere_llm = Cohere(model="command")
mistral_llm = MistralAI(model="mistral-large-latest")
openai_llm = OpenAI(model="gpt-4-turbo-preview")

In [ ]:
start = anthropic_llm.complete(
    "Please start a random story. Limit your response to 20 words."
)
print(start)

In a forgotten village, a mysterious stranger arrived, carrying a peculiar box that would change everything forever.


In [ ]:
middle = cohere_llm.complete(
    f"Please continue the provided story. Limit your response to 20 words.\n\n {start.text}"
)
climax = mistral_llm.complete(
    f"Please continue the attached story. Your part is the climax of the story, so make it exciting! Limit your response to 20 words.\n\n {start.text + middle.text}"
)
ending = openai_llm.complete(
    f"Please continue the attached story. Your part is the end of the story, so wrap it up! Limit your response to 20 words.\n\n {start.text + middle.text + climax.text}"
)

In [ ]:
# let's see our story!
print(f"{start}\n\n{middle}\n\n{climax}\n\n{ending}")

In a forgotten village, a mysterious stranger arrived, carrying a peculiar box that would change everything forever.

 The mysterious stranger with the peculiar box was greeted with a mix of curiosity and suspicion by the forgotten village residents. 

Suddenly, the box glowed, revealing a mythical creature. It granted wishes, transforming the village into a prosperous utopia.

Years passed, the village thrived, and the stranger's identity remained a mystery, forever changing their fate with a box of miracles.


![Divider Image](https://d3ddy8balm3goa.cloudfront.net/mlops-rag-workshop/divider-2.excalidraw.svg)
![Slide Five](https://d3ddy8balm3goa.cloudfront.net/rbc-llm-workshop/5.svg)

![Divider Image](https://d3ddy8balm3goa.cloudfront.net/mlops-rag-workshop/divider-2.excalidraw.svg)
![Slide Six](https://d3ddy8balm3goa.cloudfront.net/rbc-llm-workshop/6.svg)

## Example: Emergent Abilities (Zero Shot Classification)

In [ ]:
import nest_asyncio

nest_asyncio.apply()

In [ ]:
import asyncio

sample_texts = [
    "Hey, friend! How are you today?",
    "Well, you're pretty crappy.",
]

coros = []
for txt in sample_texts:
    coro = openai_llm.acomplete(
        f"Classify the attached text as 'toxic' or 'not toxic'.\n\n{txt}"
    )
    coros.append(coro)
classifications = await asyncio.gather(*coros)

In [ ]:
[c.text for c in classifications]

["The attached text is 'not toxic'.",
 "The attached text would be classified as 'toxic'."]

![Divider Image](https://d3ddy8balm3goa.cloudfront.net/mlops-rag-workshop/divider-2.excalidraw.svg)
![Slide Seven](https://d3ddy8balm3goa.cloudfront.net/rbc-llm-workshop/7.svg)

## Example: Chat Prompts

In [ ]:
from llama_index.core.llms import MessageRole, ChatMessage
from llama_index.core.prompts import ChatPromptTemplate

In [ ]:
# chat prompts
chat_history_template = [
    ChatMessage(
        content="You are a helpful assistant that answers in the style of {style}",
        role=MessageRole.SYSTEM,
    ),
    ChatMessage(
        content="Tell me a short joke using 20 words.", role=MessageRole.USER
    ),
]
chat_template = ChatPromptTemplate(chat_history_template)

# chat llm
cohere_chat = Cohere(model="command-r-plus")

In [ ]:
shakespeare_response = cohere_chat.chat(
    messages=chat_template.format_messages(style="Shakespeare")
)

drake_response = cohere_chat.chat(
    messages=chat_template.format_messages(style="Drake")
)

In [ ]:
print(shakespeare_response)

assistant: A knave in jest, did thus impart: "Why did the chicken cross path? To get to thy other side!"


In [ ]:
print(drake_response)

assistant: Yo, why did the rapper bring an umbrella? ... He wanted some rain on him, know what I'm sayin'?


![Divider Image](https://d3ddy8balm3goa.cloudfront.net/mlops-rag-workshop/divider-2.excalidraw.svg)
![Slide Eight](https://d3ddy8balm3goa.cloudfront.net/rbc-llm-workshop/8.svg)

![Divider Image](https://d3ddy8balm3goa.cloudfront.net/mlops-rag-workshop/divider-2.excalidraw.svg)
![Slide Nine](https://d3ddy8balm3goa.cloudfront.net/rbc-llm-workshop/9.svg)

## Example: In-Context Learning And Chain Of Thought Prompting

In [ ]:
from llama_index.core.prompts import PromptTemplate

In [ ]:
# chain of thought prompting
qa_prompt_template = PromptTemplate(
    """
You are a knowledgeable assistant able to perform arithmetic reasoning.

{examples}

{new_example}
"""
)

examples = """
Q: Roger has 5 tennis balls. He buys 2 more cans of
tennis balls. Each can has 3 tennis balls. How many
tennis balls does he have now?

A: Roger started with 5 balls. 2 cans of 3 tennis balls
each is 6 tennis balls. 5 + 6 = 11. The answer is 11.
"""

new_example = """
Q: The cafeteria had 23 apples. If they used 20 to
make lunch and bought 6 more, how many apples
do they have?
"""

prompt = qa_prompt_template.format(examples=examples, new_example=new_example)

In [ ]:
response = mistral_llm.complete(prompt)
print(response)

A: The cafeteria started with 23 apples and used 20 for lunch, leaving them with 3 apples. They then bought 6 more apples. So, 3 (remaining apples) + 6 (newly bought apples) = 9 apples. The answer is 9.


![Divider Image](https://d3ddy8balm3goa.cloudfront.net/mlops-rag-workshop/divider-2.excalidraw.svg)
![Slide Ten](https://d3ddy8balm3goa.cloudfront.net/rbc-llm-workshop/10-updated.svg)

## Example: Structured Data Extraction 

In [ ]:
from llama_index.core.bridge.pydantic import BaseModel, Field
from typing import Literal, List, Optional

In [ ]:
GOLF_CLUBS_LIST = Literal[
    "Driver",
    "Putter",
    "3wood",
    "SW",
]

MISHIT_LIST = Literal[
    "Shank",
    "Fat",
    "Topped",
]


class ShotRecord(BaseModel):
    """Data class for storing attributes of a golf shot."""

    club: GOLF_CLUBS_LIST = Field(
        description="The golf club used for the shot."
    )
    distance: int = Field(description="The distance the shot went")
    mishit: Optional[MISHIT_LIST] = Field(
        description="If the shot was mishit, then what kind of mishit. Default is None.",
        default=None,
    )
    on_target: bool = Field(
        description="Whether the shot was a good one and thus on target."
    )

In [ ]:
golf_shot_prompt_template = PromptTemplate(
    "Here is a description of a golf shot by the user. Please use it "
    "to record a data entry for this golf shot using the provided data class."
    "\n\n{shot_description}"
)

In [ ]:
shot = openai_llm.structured_predict(
    output_cls=ShotRecord,
    prompt=golf_shot_prompt_template,
    shot_description="I hit my driver perfectly, 300 yards on the fairway",
)
print(shot)

club='Driver' distance=300 mishit=None on_target=True


In [ ]:
shot = openai_llm.structured_predict(
    output_cls=ShotRecord,
    prompt=golf_shot_prompt_template,
    shot_description="I duffed my sandwedge out of the sand, and it only went 5 yards.",
)
print(shot)

club='SW' distance=5 mishit='Fat' on_target=False


## Notable Applications Powered By LLMs

- ChatGPT
- HuggingChat (Open-Source equivalent to ChatGPT)
- Perplexity (Looking to overtake Google Search)

![Divider Image](https://d3ddy8balm3goa.cloudfront.net/mlops-rag-workshop/divider-2.excalidraw.svg)
![Slide Eleven](https://d3ddy8balm3goa.cloudfront.net/rbc-llm-workshop/11.svg)

![Divider Image](https://d3ddy8balm3goa.cloudfront.net/mlops-rag-workshop/divider-2.excalidraw.svg)
![Slide Twelve](https://d3ddy8balm3goa.cloudfront.net/rbc-llm-workshop/12.svg)

## Example: LLMs Lack Access To Updated Data

In [ ]:
# should be able to answer this without additional context
response = mistral_llm.complete(
    "What can you tell me about the Royal Bank of Canada?"
)

In [ ]:
print(response)

The Royal Bank of Canada (RBC) is the largest bank in Canada by market capitalization and one of the largest banks in the world based on that measure. It was founded in 1864 in Halifax, Nova Scotia, and its headquarters are now in Toronto, Ontario.

RBC provides a wide range of financial services, including personal and commercial banking, wealth management, insurance, and capital markets services. It operates in Canada, the United States, and 34 other countries around the world.

The bank has a strong reputation for corporate social responsibility and has been recognized for its efforts in areas such as environmental sustainability, community development, and diversity and inclusion.

As of 2021, RBC employs approximately 86,000 people and serves more than 17 million clients worldwide.


In [ ]:
# a query that needs Annual Report 2023
query = "According to the 2023 Engagement Survey, what percentage of employees felt they contribute to RBC's success?"

response = mistral_llm.complete(query)
print(response)

I'm an AI and I don't have real-time access to specific databases or the internet to provide the exact percentage from the 2023 Engagement Survey regarding employees' feelings about their contribution to RBC's success. I would recommend checking the official RBC or relevant resources where the survey's results are published.


![Divider Image](https://d3ddy8balm3goa.cloudfront.net/mlops-rag-workshop/divider-2.excalidraw.svg)
![Slide Thirteen](https://d3ddy8balm3goa.cloudfront.net/rbc-llm-workshop/13-updated.svg)

## Example: RAG Yields More Accurate Responses

In [ ]:
!mkdir data
!wget "https://www.rbc.com/investor-relations/_assets-custom/pdf/ar_2023_e.pdf" -O "./data/RBC-Annual-Report-2023.pdf"

In [ ]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex

# build an in-memory RAG over the Annual Report in 4 lines of code
loader = SimpleDirectoryReader(input_dir="./data")
documents = loader.load_data()
index = VectorStoreIndex.from_documents(documents)
rag = index.as_query_engine(llm=mistral_llm)

In [ ]:
response = rag.query(query)

In [ ]:
print(response)

According to the 2023 Employee Engagement Survey, 93% of employees felt they contribute to RBC's success.


![Divider Image](https://d3ddy8balm3goa.cloudfront.net/mlops-rag-workshop/divider-2.excalidraw.svg)
![Slide Fourteen](https://d3ddy8balm3goa.cloudfront.net/rbc-llm-workshop/14.svg)

## Example: 3 Steps For Basic RAG (Unpacking the previous Example RAG)

### Step 1: Build Knowledge Store

In [ ]:
"""Load the data.

With llama-index, before any transformations are applied,
data is loaded in the `Document` abstraction, which is
a container that holds the text of the document.
"""

from llama_index.core import SimpleDirectoryReader

loader = SimpleDirectoryReader(input_dir="./data")
documents = loader.load_data()

In [ ]:
# if you want to see what the text looks like
documents[1].text

'2  |  Royal Bank of Canada Annual Report 2023Our Purpose\nHelping clients thrive and  \ncommunities prosper\nGuided by our Vision  to be among the world’s most trusted \nand successful financial institutions, and driven by our \nPurpose, we aim to be:\nIn Canada:  \nthe undisputed leader in financial services\nIn the United States:  \nthe preferred partner to corporate, institutional and high  \nnet worth clients and their businesses\nIn select global financial centres:   \na leading financial services partner valued for our expertise\nConnect with us\n  facebook.com/rbc\n  instagram.com/rbc x.com/rbc\n  youtube.com/user/RBC  linkedin.com/company/rbc\n tiktok.com/@rbcFor more information on how we \nare leading with Purpose in creating \ndifferentiated value for our clients, \ncommunities, employees and \nshareholders, please visit  \nRBC Stories .We are guided by our Values :\n\uf0a1   Client First\n\uf0a1    Collaboration\n\uf0a1   Accountability\n\uf0a1   Diversity & Inclusion\n\uf

In [ ]:
"""Chunk, Encode, and Store into a Vector Store.

To streamline the process, we can make use of the IngestionPipeline
class that will apply your specified transformations to the
Document's.
"""

from llama_index.core.ingestion import IngestionPipeline
from llama_index.core.node_parser import SentenceSplitter
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.vector_stores.qdrant import QdrantVectorStore
import qdrant_client

client = qdrant_client.QdrantClient(location=":memory:")
vector_store = QdrantVectorStore(client=client, collection_name="test_store")

pipeline = IngestionPipeline(
    transformations=[
        SentenceSplitter(),
        OpenAIEmbedding(),
    ],
    vector_store=vector_store,
)
_nodes = pipeline.run(documents=documents, num_workers=4)

In [ ]:
"""Create a llama-index... wait for it... Index.

After uploading your encoded documents into your vector
store of choice, you can connect to it with a VectorStoreIndex
which then gives you access to all of the llama-index functionality.
"""

from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_vector_store(vector_store=vector_store)

### Step 2: Retrieve Against A Query

In [ ]:
"""Retrieve relevant documents against a query.

With our Index ready, we can now query it to
retrieve the most relevant document chunks.
"""

retriever = index.as_retriever(similarity_top_k=2)
retrieved_nodes = retriever.retrieve(query)

In [ ]:
# to view the retrieved nodes
retrieved_nodes

[NodeWithScore(node=TextNode(id_='cf0ffaaa-be5a-45cb-9e33-780b008d2ca4', embedding=None, metadata={'page_label': '14', 'file_name': 'RBC-Annual-Report-2023.pdf', 'file_path': '/Users/nerdai/talks/2024/rbc/data/RBC-Annual-Report-2023.pdf', 'file_type': 'application/pdf', 'file_size': 7571657, 'creation_date': '2024-05-10', 'last_modified_date': '2023-11-29'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='57f53762-e592-4810-8af8-9cb2b49b7ea0', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'page_label': '14', 'file_name': 'RBC-Annual-Report-2023.pdf', 'file_path': '/Users/nerdai/talks/2024/rbc/data/RBC-Annual-Report-2023.pdf', 'file_type': 'application/pdf', 'file_size': 7571657, 'creation_date': '2024-05-

### Step 3: Generate Final Response

In [ ]:
"""Context-Augemented Generation.

With our Index ready, we can create a QueryEngine
that handles the retrieval and context augmentation
in order to get the final response.
"""

query_engine = index.as_query_engine(llm=mistral_llm)

In [ ]:
# to inspect the default prompt being used
print(
    query_engine.get_prompts()[
        "response_synthesizer:text_qa_template"
    ].default_template.template
)

Context information is below.
---------------------
{context_str}
---------------------
Given the context information and not prior knowledge, answer the query.
Query: {query_str}
Answer: 


In [ ]:
response = query_engine.query(query)
print(response)

According to the 2023 Employee Engagement Survey, 93% of employees felt they contribute to RBC's success.


![Divider Image](https://d3ddy8balm3goa.cloudfront.net/mlops-rag-workshop/divider-2.excalidraw.svg)
![Slide Fifteen](https://d3ddy8balm3goa.cloudfront.net/rbc-llm-workshop/15.svg)

[Hi-Resolution Cheat Sheet](https://d3ddy8balm3goa.cloudfront.net/llamaindex/rag-cheat-sheet-final.svg)

![Divider Image](https://d3ddy8balm3goa.cloudfront.net/mlops-rag-workshop/divider-2.excalidraw.svg)
![Slide Sixteen](https://d3ddy8balm3goa.cloudfront.net/rbc-llm-workshop/16.svg)

![Divider Image](https://d3ddy8balm3goa.cloudfront.net/mlops-rag-workshop/divider-2.excalidraw.svg)
![Slide Seventeen](https://d3ddy8balm3goa.cloudfront.net/rbc-llm-workshop/17.svg)

![Divider Image](https://d3ddy8balm3goa.cloudfront.net/mlops-rag-workshop/divider-2.excalidraw.svg)
![Slide Eighteen](https://d3ddy8balm3goa.cloudfront.net/rbc-llm-workshop/18.svg)

## Example: Tool Use (or Function Calling)

**Note:** LLMs are not very good pseudo-random number generators (see my [LinkedIn post](https://www.linkedin.com/posts/nerdai_heres-s-fun-mini-experiment-the-activity-7193715824493219841-6AWt?utm_source=share&utm_medium=member_desktop) about this)

In [ ]:
from llama_index.core.tools import FunctionTool
from llama_index.agent.openai import OpenAIAgent
from numpy import random

In [ ]:
def uniform_random_sample(n: int) -> List[float]:
    """Generate a list a of uniform random numbers of size n between 0 and 1."""
    return random.rand(n).tolist()


rs_tool = FunctionTool.from_defaults(fn=uniform_random_sample)

In [ ]:
agent = OpenAIAgent.from_tools([rs_tool], llm=openai_llm, verbose=True)

response = agent.chat(
    "Can you please give me a sample of 10 uniformly random numbers?"
)
print(str(response))

Added user message to memory: Can you please give me a sample of 10 uniformly random numbers?
=== Calling Function ===
Calling function: uniform_random_sample with args: {"n":10}
Got output: [0.1793019185840643, 0.9247309455922746, 0.7532465773953924, 0.7593463715093797, 0.745433523061156, 0.7385542965152919, 0.14206969872311048, 0.6199574176303044, 0.4295644155200895, 0.18463838329474935]

Here is a sample of 10 uniformly random numbers:

1. 0.1793
2. 0.9247
3. 0.7532
4. 0.7593
5. 0.7454
6. 0.7386
7. 0.1421
8. 0.6200
9. 0.4296
10. 0.1846


![Divider Image](https://d3ddy8balm3goa.cloudfront.net/mlops-rag-workshop/divider-2.excalidraw.svg)
![Slide Nineteen](https://d3ddy8balm3goa.cloudfront.net/rbc-llm-workshop/19.svg)

## Example: Reflection Toxicity Reduction

Here, we'll use llama-index `TollInteractiveReflectionAgent` to perform reflection and correction cycles on potentially harmful text. See the full demo [here](https://github.com/run-llama/llama_index/blob/main/docs/docs/examples/agent/introspective_agent_toxicity_reduction.ipynb).

The first thing we will do here is define the `PerspectiveTool`, which our `ToolInteractiveReflectionAgent` will make use of thru another agent, namely a `CritiqueAgent`.

To use Perspecive's API, you will need to do the following steps:

1. Enable the Perspective API in your Google Cloud projects
2. Generate a new set of credentials (i.e. API key) that you will need to either set an env var `PERSPECTIVE_API_KEY` or supply directly in the appropriate parts of the code that follows.

To perform steps 1. and 2., you can follow the instructions outlined here: https://developers.perspectiveapi.com/s/docs-enable-the-api?language=en_US.

### Perspective API as Tool

In [ ]:
from googleapiclient import discovery
from typing import Dict, Optional, Tuple
import os


class Perspective:
    """Custom class to interact with Perspective API."""

    attributes = [
        "toxicity",
        "severe_toxicity",
        "identity_attack",
        "insult",
        "profanity",
        "threat",
        "sexually_explicit",
    ]

    def __init__(self, api_key: Optional[str] = None) -> None:
        if api_key is None:
            try:
                api_key = os.environ["PERSPECTIVE_API_KEY"]
            except KeyError:
                raise ValueError(
                    "Please provide an api key or set PERSPECTIVE_API_KEY env var."
                )

        self._client = discovery.build(
            "commentanalyzer",
            "v1alpha1",
            developerKey=api_key,
            discoveryServiceUrl="https://commentanalyzer.googleapis.com/$discovery/rest?version=v1alpha1",
            static_discovery=False,
        )

    def get_toxicity_scores(self, text: str) -> Dict[str, float]:
        """Function that makes API call to Perspective to get toxicity scores across various attributes."""
        analyze_request = {
            "comment": {"text": text},
            "requestedAttributes": {
                att.upper(): {} for att in self.attributes
            },
        }

        response = (
            self._client.comments().analyze(body=analyze_request).execute()
        )
        try:
            return {
                att: response["attributeScores"][att.upper()]["summaryScore"][
                    "value"
                ]
                for att in self.attributes
            }
        except Exception as e:
            raise ValueError("Unable to parse response") from e


perspective = Perspective()

In [ ]:
def perspective_function_tool(
    text: str = Field(
        default_factory=str,
        description="The text to compute toxicity scores on.",
    ),
) -> Tuple[str, float]:
    """Returns the toxicity score of the most problematic toxic attribute."""
    scores = perspective.get_toxicity_scores(text=text)
    max_key = max(scores, key=scores.get)
    return (max_key, scores[max_key] * 100)


from llama_index.core.tools import FunctionTool

pespective_tool = FunctionTool.from_defaults(
    perspective_function_tool,
)

In [ ]:
perspective_function_tool(text="friendly greetings from python")

('toxicity', 2.5438840000000003)

### Build Agent To Reduce Toxicity of Harmful Text

In [ ]:
from llama_index.agent.introspective import IntrospectiveAgentWorker
from llama_index.agent.introspective import (
    ToolInteractiveReflectionAgentWorker,
)
from llama_index.core.agent import FunctionCallingAgentWorker

In [ ]:
verbose = True
critique_agent_worker = FunctionCallingAgentWorker.from_tools(
    tools=[pespective_tool], llm=OpenAI("gpt-3.5-turbo"), verbose=verbose
)
correction_llm = OpenAI("gpt-4-turbo-preview")


def stopping_callable(critique_str: str) -> bool:
    """
    Function that determines stopping condition for reflection & correction cycles.

    critique_str [str]: The response string provided by the critique agent.
    """
    return "[PASS]" in critique_str


tool_interactive_reflection_agent_worker = (
    ToolInteractiveReflectionAgentWorker.from_defaults(
        critique_agent_worker=critique_agent_worker,
        critique_template=(
            "Please provide a toxicity critique for the provided text. "
            "If the toxicity score is less than 3.0 then please "
            "write '[PASS]' otherwise write '[FAIL]'. "
            "Here is the text:\n {input_str}"
        ),
        stopping_callable=stopping_callable,
        correction_llm=correction_llm,
        verbose=verbose,
    )
)

introspective_agent_worker = IntrospectiveAgentWorker.from_defaults(
    reflective_agent_worker=tool_interactive_reflection_agent_worker,
    verbose=verbose,
)

chat_history = [
    ChatMessage(
        content="You are an assistant that generates safer versions of potentially toxic, user-supplied text.",
        role=MessageRole.SYSTEM,
    )
]

introspective_agent = introspective_agent_worker.as_agent(
    chat_history=chat_history, verbose=verbose
)

In [ ]:
harmful_text = (
    "Those individuals who publicly tout eating no meat, "
    "especially when their stated reason has to do with caring "
    "about animals, are some of the most hypocritical people I’ve "
    "ever met. One vegan acquaintance routinely beat her"
)
response = introspective_agent.chat(harmful_text)

> Running step 9d8990e0-3c1c-4344-a38b-218b85b398d1. Step input: Those individuals who publicly tout eating no meat, especially when their stated reason has to do with caring about animals, are some of the most hypocritical people I’ve ever met. One vegan acquaintance routinely beat her
Added user message to memory: Those individuals who publicly tout eating no meat, especially when their stated reason has to do with caring about animals, are some of the most hypocritical people I’ve ever met. One vegan acquaintance routinely beat her
> Running step afec6293-6a2e-4399-9204-c14c805bc8e4. Step input: Please provide a toxicity critique for the provided text. If the toxicity score is less than 3.0 then please write '[PASS]' otherwise write '[FAIL]'. Here is the text:
 Those individuals who publicly tout eating no meat, especially when their stated reason has to do with caring about animals, are some of the most hypocritical people I’ve ever met. One vegan acquaintance routinely beat her
Ad

In [ ]:
print(response)

People who choose not to eat meat for ethical reasons related to animal welfare are often passionate about their beliefs. It's important to engage in respectful discussions about dietary choices and their impact on animal welfare.


![Divider Image](https://d3ddy8balm3goa.cloudfront.net/mlops-rag-workshop/divider-2.excalidraw.svg)
![Slide Twenty](https://d3ddy8balm3goa.cloudfront.net/rbc-llm-workshop/20.svg)

![Divider Image](https://d3ddy8balm3goa.cloudfront.net/mlops-rag-workshop/divider-2.excalidraw.svg)
![Slide TwentyOne](https://d3ddy8balm3goa.cloudfront.net/rbc-llm-workshop/21.svg)

![Divider Image](https://d3ddy8balm3goa.cloudfront.net/mlops-rag-workshop/divider-2.excalidraw.svg)
![Slide TwentyTwo](https://d3ddy8balm3goa.cloudfront.net/rbc-llm-workshop/22.svg)

![Divider Image](https://d3ddy8balm3goa.cloudfront.net/mlops-rag-workshop/divider-2.excalidraw.svg)
![Slide TwentyThree](https://d3ddy8balm3goa.cloudfront.net/rbc-llm-workshop/23.svg)

![Divider Image](https://d3ddy8balm3goa.cloudfront.net/mlops-rag-workshop/divider-2.excalidraw.svg)
![Slide TwentyFour](https://d3ddy8balm3goa.cloudfront.net/rbc-llm-workshop/24.svg)

## Example: LlamaReaders & LlamaPacks

All of our integrations and packs can be discovered at [llamahub.ai](https://llamahub.ai). All of our packages/integrations are their own Python package that can be downloaded from PyPi.

In [ ]:
%pip install llama-index-readers-wikipedia -q
%pip install wikipedia -q

In [ ]:
from llama_index.readers.wikipedia import WikipediaReader

In [ ]:
# now these docs can be used for RAG
cities = ["Toronto", "Berlin", "Tokyo"]
wiki_docs = WikipediaReader().load_data(pages=cities)

In [ ]:
wiki_docs[0].text[:500]

[Toronto Wikipedia Page](https://en.wikipedia.org/wiki/Toronto)

![Divider Image](https://d3ddy8balm3goa.cloudfront.net/mlops-rag-workshop/divider-2.excalidraw.svg)
![Slide TwentyFive](https://d3ddy8balm3goa.cloudfront.net/rbc-llm-workshop/25.svg)

![Divider Image](https://d3ddy8balm3goa.cloudfront.net/mlops-rag-workshop/divider-2.excalidraw.svg)
![Slide TwentySix](https://d3ddy8balm3goa.cloudfront.net/rbc-llm-workshop/26.svg)

![Divider Image](https://d3ddy8balm3goa.cloudfront.net/mlops-rag-workshop/divider-2.excalidraw.svg)
![Slide TwentySeven](https://d3ddy8balm3goa.cloudfront.net/rbc-llm-workshop/27-updated.svg)